Law of large numbers

In [4]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
iris=load_iris() #데이터셋 로드
X=iris.data[:,2:] #꽃잎의 길이, 너비
Y=iris.target
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True)
#shuffle=True: 데이터 편향 방지, 일반화 성능 향상 (shuffle을 주지 않으면 y데이터가 0000, 1111, 2222로 자동 sort되어있어 섞어줘야함)

In [6]:
#약한 학습기(weak learner) 구축
log_model = LogisticRegression() #로지스틱 회귀 모델 생성
rnd_model = RandomForestClassifier() #랜덤 포레스트 분류기 생성
svm_model = SVC() #서포트 벡터 머신 분류기 생성

In [7]:
#3개의 약한 학습기들을 결합하여 앙상블 모델 생성
voting_model = VotingClassifier(
    estimators=[('lr',log_model),('rf',rnd_model),('svc',svm_model)],
    voting='hard'
)

#estimators: 앙상블에 포함될 개별 모델들 지정
#voting: hard 투표 방식. 개별 모델의 예측값들을 다수결로 결정

In [8]:
#앙상블 모델 학습
voting_model.fit(x_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [9]:
for model in (log_model, rnd_model, svm_model, voting_model):
  model.fit(x_train,y_train)
  y_pred = model.predict(x_test) #학습된 모델을 사용해 테스트데이터(x_test)에 대한 예측을 수행
  print(model.__class__.__name__," : ",accuracy_score(y_test,y_pred))

#model.__class__.__name__: 각 모델 객체의 클래스 이름. 해당 모델이 테스트데이터에 대해 얼마나 잘 예측하는지 측정 후 정확도 출력
#accuracy_score: 모델의 예측값과 실제값을 비교하여 정확도를 계산하는 함수

LogisticRegression  :  1.0
RandomForestClassifier  :  0.9555555555555556
SVC  :  1.0
VotingClassifier  :  1.0


앙상블모델 - 사이킷런의 배깅과 페이스팅

In [10]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [11]:
bag_model = BaggingClassifier(
    DecisionTreeClassifier(), #약한 학습기(결정 트리)
    n_estimators=500, #약한 학습기(결정 트리) 500개 생성 (랜덤포레스트와 유사)
    max_samples=0.05, #0.0~1.0 사이 실수 선택 가능
    bootstrap=True, #True: 배깅(중복 허용), False: 페이스팅(중복 X)
    n_jobs=-1 #훈련과 예측에 사용할 CPU 코어 수 결정(-1: 가용한 모든 코어 사용)
)    

In [12]:
iris = load_iris()
X = iris.data[:,2:]
Y = iris.target
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True) 

In [13]:
#모델 학습
bag_model.fit(x_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.05,
                  n_estimators=500, n_jobs=-1)

In [14]:
#모델 예측
y_pred = bag_model.predict(x_test)

#모델 평가
print(bag_model.__class__.__name__," : ",accuracy_score(y_test,y_pred))

BaggingClassifier  :  0.9777777777777777


oob(out-of-bag) 평가

In [15]:
#배깅(중복 허용 샘플링)을 하다보면 평균적으로 훈련 샘플의 약 63%만 추출되고 나머지 37%는 추출되지 않음
#추출되지 않은 샘플들 = oob(out-of-bag)
#앙상블의 평가는 각 예측기의 oob평가의 평균으로 확인

In [16]:
iris = load_iris()
X = iris.data[:,2:]
Y = iris.target
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True) 

In [17]:
#모델 구축. BaggingClassifier를 사용하여 결정 트리 분류기 생성
bag_model = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 500, #앙상블에 포함될 결정 트리의 개수 지정
    bootstrap = True, #데이터를 중복할지 여부
    n_jobs = -1, #사용할 CPU 코어 수. -1로 설정 시 가능한 모든 코어를 사용
    oob_score = True #oob샘플을 사용한 평가를 활성화
)

#모델 학습
bag_model.fit(x_train,y_train)

#모델 평가(oob_score_)
print('oob_score : ',bag_model.oob_score_)

#모델 평가
y_pred = bag_model.predict(x_test)
print('test_score : ',accuracy_score(y_test,y_pred))

oob_score :  0.9523809523809523
test_score :  0.9333333333333333


랜덤포레스트

In [18]:
#배깅방법을 사용한 결정트리 앙상블 모델. BaggingClassifier에 DecisionTreeClassifier를 넣는 대신, RandomForestClassifier를 사용
#무작위성. 노드 분할 시 전체 특성 중에서 최선의 특성을 찾는 것이 아니라, 무작위로 선택한 들성들 중에서 최선의 특성을 찾는 방식

In [19]:
from sklearn.ensemble import RandomForestClassifier

#랜덤포레스트 모델 구축
rnd_model = RandomForestClassifier(
    n_estimators = 500, #500개의 결정 트리
    max_leaf_nodes = 16, #각 트리는 최대 16개의 잎 노드를 가짐
    n_jobs = -1 #가능한 모든 CPU 코어 사용
)

#모델 학습
rnd_model.fit(x_train,y_train)

#모델 예측
y_pred_rf = rnd_model.predict(x_test)

#모델 평가
print("rnd_model : ",accuracy_score(y_pred_rf,y_test))

rnd_model :  0.9333333333333333


In [25]:
#특성 중요도

x = iris.data[:,:]
y = iris.target

rnd_model = RandomForestClassifier(
    n_estimators = 500,
    n_jobs = -1
)

rnd_model.fit(x,y)

for feature_name,feature_imp in zip(iris['feature_names'],rnd_model.feature_importances_):
  print(feature_name,' : ',feature_imp)

sepal length (cm)  :  0.1118235966685349
sepal width (cm)  :  0.025567682987568733
petal length (cm)  :  0.41559035057779237
petal width (cm)  :  0.44701836976610393


에이다부스트

In [26]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_model = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators = 200,
    algorithm = 'SAMME.R',
    learning_rate=0.5
)

# 모델 학습
ada_model.fit(x,y)

#부스팅: 약한 학습기를 여러 개 연결하여 순차적으로 보완하면서 강력한 학습기를 만드는 앙상블 방법

c:\Users\tjoeun\miniconda3\envs\firstenv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)